In [21]:
import pandas as pd
from sklearn.decomposition import TruncatedSVD

In [22]:
df= pd.read_csv("datasetreco.csv", sep=";")
df.head()

,Date,Time,event_type,user_id,user_session,product_id,price,category,subcategory,subsubcategory,is_view,is_cart,is_purchase,p_views,p_carts,p_purchases,user_views,user_carts,user_purchases
0,2020-09-24,12:02:19,view,1515915625519390366,F71OG8wWVP,3828758,21.59,electronics,audio,acoustic,1,0,0,372,1,0,1,0,0
1,2020-09-24,12:22:49,view,1515915625519398031,hq3JGndGca,1271549,405.65,electronics,audio,acoustic,1,0,0,365,1,0,1,0,0
2,2020-09-24,12:48:31,view,1515915625519407618,JB3Sy5LCfb,124712,204.79,electronics,audio,acoustic,1,0,0,513,68,61,2,1,1
3,2020-09-24,12:49:34,view,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,1,0,0,513,68,61,2,1,1
4,2020-09-24,12:50:22,cart,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,0,1,0,513,68,61,2,1,1


In [23]:
def rating(df):
    coef_view,coef_cart,coef_purchase= 1, 5, 10
    df["rating"]=df["p_views"]*coef_view + df["p_carts"]*coef_cart+df["p_purchases"]*coef_purchase
    return df



In [24]:
df=rating(df)
df.head()

,Date,Time,event_type,user_id,user_session,product_id,price,category,subcategory,subsubcategory,is_view,is_cart,is_purchase,p_views,p_carts,p_purchases,user_views,user_carts,user_purchases,rating
0,2020-09-24,12:02:19,view,1515915625519390366,F71OG8wWVP,3828758,21.59,electronics,audio,acoustic,1,0,0,372,1,0,1,0,0,377
1,2020-09-24,12:22:49,view,1515915625519398031,hq3JGndGca,1271549,405.65,electronics,audio,acoustic,1,0,0,365,1,0,1,0,0,370
2,2020-09-24,12:48:31,view,1515915625519407618,JB3Sy5LCfb,124712,204.79,electronics,audio,acoustic,1,0,0,513,68,61,2,1,1,1463
3,2020-09-24,12:49:34,view,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,1,0,0,513,68,61,2,1,1,1463
4,2020-09-24,12:50:22,cart,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,0,1,0,513,68,61,2,1,1,1463


In [25]:
df_group= df[df["p_purchases"]>= 1]
df_group.head()

,Date,Time,event_type,user_id,user_session,product_id,price,category,subcategory,subsubcategory,is_view,is_cart,is_purchase,p_views,p_carts,p_purchases,user_views,user_carts,user_purchases,rating
2,2020-09-24,12:48:31,view,1515915625519407618,JB3Sy5LCfb,124712,204.79,electronics,audio,acoustic,1,0,0,513,68,61,2,1,1,1463
3,2020-09-24,12:49:34,view,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,1,0,0,513,68,61,2,1,1,1463
4,2020-09-24,12:50:22,cart,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,0,1,0,513,68,61,2,1,1,1463
5,2020-09-24,12:51:06,purchase,1515915625519407618,bRZAHDpYFC,124712,204.79,electronics,audio,acoustic,0,0,1,513,68,61,2,1,1,1463
9,2020-09-24,13:54:46,view,1515915625519431986,4DPxT81h91,4100847,31.27,electronics,audio,acoustic,1,0,0,229,18,15,1,0,0,469


In [26]:
df_reduct= df_group[["user_id","product_id","subsubcategory","rating"]]
df_reduct.head()

,user_id,product_id,subsubcategory,rating
2,1515915625519407618,124712,acoustic,1463
3,1515915625519407618,124712,acoustic,1463
4,1515915625519407618,124712,acoustic,1463
5,1515915625519407618,124712,acoustic,1463
9,1515915625519431986,4100847,acoustic,469


In [27]:
df_reduct= df_reduct.drop_duplicates()

In [28]:
max_ratings = df_reduct.groupby('subsubcategory')['rating'].transform('max')
df_reduct['normalized_rating'] = df_reduct['rating'] / max_ratings
df_reduct.head()

,user_id,product_id,subsubcategory,rating,normalized_rating
2,1515915625519407618,124712,acoustic,1463,0.951854
9,1515915625519431986,4100847,acoustic,469,0.305140
10,1515915625482978862,1674137,acoustic,33,0.021470
11,1515915625517146957,1166287,acoustic,201,0.130774
15,1515915625519470612,857266,acoustic,58,0.037736


In [29]:
df_pivot = df_reduct.pivot(index='product_id', columns='user_id', values='rating')
df_pivot= df_pivot.fillna(0)
df_pivot.head()

user_id,1515915625353230683,1515915625353230922,1515915625353234047,1515915625353286099,1515915625353294441,1515915625353457259,1515915625353458744,1515915625353496458,1515915625353555928,1515915625353561691,...,1515915625611017481,1515915625611017996,1515915625611018038,1515915625611018641,1515915625611018705,1515915625611020601,1515915625611022256,1515915625611022570,1515915625611022696,1515915625611023581
product_id,,,,,,,,,,,,,,,,,,,,,
2365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3727,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
dimention=10
svd=TruncatedSVD(n_components=dimention)
df_pivot_trans=svd.fit_transform(df_pivot)

